In [13]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [14]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import reload_lib as rl
import non_bio_co2 as nbc


lib = [s, lc, lp, nbc]

In [15]:
from copy import deepcopy as dc

In [16]:
import bw2io as bi
import bw2data as bd
import pandas as pd

In [17]:
bd.projects.set_current("single use vs multi use")


In [18]:
if any("biosphere" in db for db in bd.databases):
    print('Biosphere is already present in the project.')
else:
    bi.bw2setup()

Biosphere is already present in the project.


In [41]:
if 'ev391cutoff' in bd.databases and 'ev391consq' in bd.databases and 'ev391apos' in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    # APOS
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets", db_name='ev391apos') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # Consequential
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets", db_name='ev391consq') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # cut-off
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets", db_name='ev391cutoff') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

Ecoinvent 3.9.1 is already present in the project.


In [59]:
system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
              r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

for path in system_path:    
    db_path = path 
    sheet_name = ['ev391apos', 'ev391consq', 'ev391cutoff'] # Replace with your actual sheet name
    for db in sheet_name:
        print(db)
    #     # Read the Excel file
    # # Read the specific sheet from the Excel file
        data = pd.read_excel(db_path, sheet_name=db)
    #     # Save the data to a temporary file that can be used by ExcelImporter
    #     temp_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\temp_use_phase.xlsx'
    #     data.to_excel(temp_path, index=False)



    #     # Use the temporary file with ExcelImporter
    #     imp = bi.ExcelImporter(temp_path)  # the path to your inventory excel file
    #     imp.apply_strategies()

    #     # # List of databases to match

    #     # Loop through each database and match
    #     print(f"Matching database: {db}")
    #     imp.match_database(db, fields=('name', 'unit', 'location', 'reference product'))
    #     # print(f"Unlinked items after matching {db}: {list(imp.unlinked)}")

    #     # Match without specifying a database
    #     imp.match_database(fields=('name', 'unit', 'location'))

    #     # Generate statistics and write results
    #     imp.statistics()
    #     imp.write_excel(only_unlinked=True)
    #     unlinked_items = list(imp.unlinked)
    #     imp.write_database()

    #     # Print unlinked items if needed
    #     print(unlinked_items)
    #     print(f'{path} is loaded into the database')

ev391apos
ev391consq
ev391cutoff
ev391apos
ev391consq
ev391cutoff


ValueError: Worksheet named 'ev391cutoff' not found